# Programming Exercise

## Part 1- File organisation

### Steps

1. Renaming files
2. Creating folders
3. Moving files to folder

In [ ]:
import scipy.io
import shutil
import os

In [ ]:
def renameFiles(directory, filesPerSubject):
    '''
    Renames files in DATA folder, replacing subject initials with subject number 
    (organised via alphabetical order)
    
    Parameters
    ----------
    directory -- string of directory pathway
    filesPerSubject -- integer; number of files for each subject

    Returns
    -------
    None

    '''
    # Retrieve files in alphabetical order
    myFiles = sorted(os.listdir(directory))
    
    # Following variable will allow for detection of new subject file, as opposed to additional file from same subject
    isNew = 1
    i = 1
    
    # Rename file according to alphabetical order
    for filename in myFiles:
        os.rename(os.path.join(directory,filename), os.path.join(directory,'00' + str(i) + str(filename[2:])))
        
        # If 'isNew' has not reached specified number of files per subject, add one
        if isNew != filesPerSubject:
            isNew += 1
            
        # If 'isNew' == number of files per subject, add 1 to i, as next file will correspond to a new subject
        else:   
            i += 1
            isNew = 1

In [ ]:
directory = ('/Users/levi/Downloads/DATA/')
filesPerSubject = 2
 
renameFiles(directory, filesPerSubject)

In [ ]:
def createFolders(parent_dir, numSubjects):
    '''
    Creates new folders to organise each subjects' data

    Parameters
    ----------
    parent_dir -- string; the parent directory in which the new folders should be created
    numSubjects -- integer; total number of subjects

    Returns
    -------
    folders -- list of strings containing name of each new folder created

    '''
    # Create list containing names of folders to be created
    folders = []
    for i in range(numSubjects):
        folders.append('00'+str(i+1))

    # For each folder in 'folders', create a new directory with this name
    for folder in folders:
        path = os.path.join(parent_dir, folder)
        os.mkdir(path)
    
    # Return list of folder names for next function
    return folders

In [ ]:
parent_dir = ('/Users/levi/Downloads/DATA/')
numSubjects = 5

folders = createFolders(parent_dir, numSubjects)

In [ ]:
def groupFiles(parent_dir, folders):
    '''
    Organise files into folders created by previous function

    Parameters
    ----------
    parent_dir : string; the parent directory in file organisation should occur
    folders : list of folder names

    Returns
    -------
    None.

    '''
    # Retrieve all files within given parent directory
    myFiles = sorted(os.listdir(parent_dir))

    for file in myFiles:
        
        # Identify the files with data (and ignore folders)
        if file.endswith('.mat'):
            sourcepath = os.path.join(parent_dir,file)
            
            # Identify which one of the folders matches with the file
            match = folders[folders.index(file[0:3])]
            
            # Set this as the destination
            destpath = parent_dir + match + '/' + file
            shutil.move(sourcepath, destpath)

In [ ]:
groupFiles(parent_dir, folders)

## Part 2 - Data visualisation.

### Steps

1. Loading files
2. Graph of all subjects' reaction times
3. Graph of all subjects' task outcome
4. Average reaction times
5. Average task outcome

In [ ]:
import os
import scipy.io
from scipy.stats import sem
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math

In [ ]:
# Load .mat files into Python IDE
directory = ('/Users/levi/Downloads/DATA/')

# Set up separate lists for task data and pupil data
task = []
pupil = []

# Iterate through each subject folder (add  [1:] here to get rid of '.DS_Store')
for folder in sorted(os.listdir(directory))[1:]:
    
    # Iterate through each file
    newDirectory = os.path.join(directory,folder)
    for file in sorted(os.listdir(newDirectory)):
        
        # If it is a task file, place it in task list
        if file.endswith('task.mat'):
            task.append(scipy.io.loadmat(os.path.join(newDirectory,file)))
            
         # If it is a pupil file, place it in task list
        if file.endswith('pupil.mat'):
            pupil.append(scipy.io.loadmat(os.path.join(newDirectory,file)))

In [ ]:
def convertToDict(matfiles):
    '''
    Extracts matrices from .mat files and places them into a dictionary, allowing for analysis within Python

    Parameters
    ----------
    matfiles - list containing .mat files

    Returns
    -------
    subjectDict - Dictionary containing matrix representation of data within each .mat file

    '''
    # Set up empty dictionary
    subjectDict = {}
    
    # Iterate over .mat files
    for i in range(len(matfiles)):
        
        # Final key in files stores the actual data structure; retrieve this
        mainKey = (list(matfiles[i].keys()))[-1]
        
        # Add it into dictionary according under key of corresponding subject
        subjectDict['00'+ str(i+1)] = matfiles[i][mainKey]
    
    return subjectDict

In [ ]:
taskDict = convertToDict(task)
print(taskDict['001'])

In [ ]:
def createGraph(subject, graphType):

    xVals = np.arange(1,41)

    yValsHard = subject[:40,2]
    yValsEasy = subject[40:,2]

    colormap = np.array(['blue','red', 'green'])

    colorsHard = subject[:40,1]
    colorsEasy = subject[40:,1]

    def cleanColors(colors):
        
        for i in range(len(colors)):
            if math.isnan(colors[i]):
                colors[i] = 2
        return colors
    
    colorsEasy = cleanColors(colorsEasy)
    colorsHard = cleanColors(colorsHard)


    def removeNaN(yVals):
        for i in range(len(yVals)):
            if math.isnan(yVals[i]):
                yVals[i] = 0
        return yVals

    yValsEasy = removeNaN(yValsEasy)
    yValsHard = removeNaN(yValsHard)

    colorsEasy = colorsEasy.astype(int)
    colorsHard = colorsHard.astype(int)

    if graphType == 'scatter':
        # #### Scatter plot representation
        plt.subplot(2,1,1)
        plt.scatter(xVals, yValsEasy, s=100, marker='<', c=colormap[colorsEasy])
        plt.subplot(2,1,2)
        plt.scatter(xVals, yValsHard, s=100, marker='<', c=colormap[colorsHard])


    elif graphType == 'bar':
        ### Bar graph representation
        plt.subplot(2,1,1)
        plt.bar(xVals, yValsEasy, color = colormap[colorsEasy])
        plt.subplot(2,1,2)
        plt.bar(xVals, yValsHard, color = colormap[colorsHard])
        

In [ ]:
createGraph(taskDict['002'], 'bar')

## Individual subject visualisation

In [ ]:
def createGraphReactions(subjectDict):
    '''
    Create grouped bar chart for each subject's average reaction time in each condition.
    
    Parameters
    ----------
    subjectDict -- Dictionary containing each subject's data

    Returns
    -------
    reactionsEasy -- list storing all five subject's avg reaction time in easy condition
         
    reactionsHard -- list storing all five subject's avg reaction time in hard condition
       
    These two lists are outputted as they will be of use for later function

    '''
    # Empty lists for average reactions
    reactionsEasy = []
    reactionsHard = []
    
    # Empty lists to store standard deviation
    stdevEasy = []
    stdevHard = []
    
    # Append each list with appropriate columns from data matrix
    for subject in list(subjectDict.values()):
        reactionsHard.append(subject[:40,2])
        reactionsEasy.append(subject[40:,2])
        
        #Append 'stdev' lists with standard deviation of specified columns
        stdevHard.append(np.nanstd(subject[:40,2]))
        stdevEasy.append(np.nanstd(subject[40:,2]))
    
    # Convert values stored in 'reaction' lists to means
    for i in range(len(reactionsEasy)):
        reactionsEasy[i] = np.nanmean(reactionsEasy[i])
        
    for i in range(len(reactionsHard)):
        reactionsHard[i] = np.nanmean(reactionsHard[i])
        
    # Set up X labels and X-axis, and plot data in grouped bar chart   
    X = ['Subject 1','Subject 2', 'Subject 3', 'Subject 4', 'Subject 5']
    X_axis = np.arange(len(X))
    plt.bar(X_axis - 0.2, reactionsEasy, 0.4, label = 'Easy')
    plt.bar(X_axis + 0.2, reactionsHard, 0.4, label = 'Hard')
    
    # Set up labels and legend
    plt.xticks(X_axis, X)
    plt.xlabel("Participant", fontsize = 13)
    plt.ylabel("Avg Reaction Time (miliseconds)", fontsize = 13)
    plt.title("Subject reaction time to cognitive task", fontsize = 15)
    plt.legend()
    
    # Use standard deviations to plot variance 
    plt.errorbar(X_axis-0.2, reactionsEasy, yerr = stdevEasy, fmt = ' ', ecolor = 'black', capsize = 5)
    plt.errorbar(X_axis+0.2, reactionsHard, yerr = stdevHard, fmt = ' ', ecolor = 'black', capsize = 5)
    plt.show()
    
    # Return list of average reaction times
    return reactionsEasy, reactionsHard

In [ ]:
reactionsEasy, reactionsHard = createGraphReactions(taskDict)

In [ ]:
def createGraphOutcomes(subjectDict):
    '''
    Create grouped stacked bar chart how many trials of the cognitive task each subject completed,
    and how many of these were correct/incorrect

    Parameters
    ----------
    subjectDict -- Dictionary containing each subject's data (same as function above)

    Returns
    -------
    correctEasy -- List; total number of correct answers for each subject in Easy condition
    correctHard -- List; total number of correct answers for each subject in Hard condition

    '''
    # Empty lists to story correct, incorrect, and missed for all subjects in each condition
    correctEasy = []
    incorrectEasy = []
    missedEasy = []
    
    correctHard = []
    incorrectHard = []
    missedHard = []
    
    # Iterate over each subject to update lists above
    for subject in list(subjectDict.values()):
        correct = 0
        incorrect = 0
        missed = 0
        
        # Start with first 40 rows (Hard condition)
        for i in subject[:40,1]:
            if i == 0:
                correct += 1
            elif i == 1:
                incorrect += 1
            else:
                missed += 1
        correctHard.append(correct)
        incorrectHard.append(incorrect)
        missedHard.append(missed)
        
        # Reset counters
        correct = 0
        incorrect = 0
        missed = 0
        
        # Second 40 rows (Easy condition)
        for i in subject[40:,1]:
            if i == 0:
                correct += 1
            elif i == 1:
                incorrect += 1
            else:
                missed += 1
        correctEasy.append(correct)
        incorrectEasy.append(incorrect)
        missedEasy.append(missed)
    
    # Set up X labels and X-axis
    X = ['Subject 1','Subject 2', 'Subject 3', 'Subject 4', 'Subject 5']
    X_axis = np.arange(len(X))
    sns.set(style = 'darkgrid')
    
    # Plot Easy condition
    plt.bar(X_axis - 0.2, correctEasy, 0.3, color = 'darkblue', alpha = 0.7)
    plt.bar(X_axis - 0.2, incorrectEasy, 0.3, bottom=correctEasy, color = 'darkred', alpha = 0.7)
    
    # Plot Hard condition
    plt.bar(X_axis + 0.2, correctHard, 0.3, color = 'darkblue', alpha = 0.7)
    plt.bar(X_axis + 0.2, incorrectHard, 0.3, bottom=correctHard, color = 'darkred', alpha = 0.7)
    
    # Add labels
    plt.xticks(X_axis, X)
    plt.ylim(top = 48)
    plt.xlabel('Participants', fontsize = 13)
    plt.ylabel("Trials completed", fontsize = 13)
    plt.title("Success ratio of each subject in Cognitive Task", fontsize = 13)
   
    # To clearly distinguish between Easy and Hard condition, labels are placed on top of each bar
    # Below are helper functions to create labels for each condition
    def addlabelsEasy(x,y):
        for i in range(len(x)):
            plt.text(x[i], y[i]+1, 'Easy', ha = 'center', fontsize = 'small')  
    
    def addlabelsHard(x,y):
        for i in range(len(x)):
            plt.text(x[i], y[i]+1, 'Hard', ha = 'center', fontsize = 'small')
            
    # X-coordinates for 'Easy' and 'Hard' labels, respectively
    X1 = X_axis - 0.2
    X2 = X_axis + 0.2
    
    # Y-coordinates for 'Easy' and 'Hard' labels, respectively
    Y1 = np.add(correctEasy, incorrectEasy)
    Y2 = np.add(correctHard, incorrectHard)
    
    # Add labels
    addlabelsEasy(X1, Y1)
    addlabelsHard(X2, Y2)
    
    # Place legend outside of figure
    plt.legend(bbox_to_anchor=(0.6, 0.55, 0.5, 0.5), labels=['Correct', 'Incorrect'], fontsize = 'x-small')
    plt.show()
    
    # Return list containing number of correct responses
    return correctEasy, correctHard

In [ ]:
correctEasy, correctHard = createGraphOutcomes(taskDict)

## Data across five participants


In [ ]:
plt.hist(reactionsEasy)
plt.show()

plt.hist(reactionsHard)
plt.show()

In [ ]:
# Set up list containing arrays of reaction times for Easy and Hard condition, respectively
data = [reactionsEasy, reactionsHard]
labels = ['Easy', 'Hard']

# Set up subplots
fig, ax = plt.subplots()
medianprops = dict(linestyle='-', linewidth=2.5, color='firebrick')
bplot = ax.boxplot(data, patch_artist = True, labels = labels, medianprops = medianprops)

# Colour each plot differently
colors = ['lightblue', 'tan']
for patch, color in zip(bplot['boxes'], colors):
    patch.set_facecolor(color)
    
# Add labels
plt.xlabel('Experimental Condition', fontsize = 13)
plt.ylabel("Reaction time (milliseconds")
plt.title("Reaction times in each condition", fontsize = 15)
plt.show()

In [ ]:
def getAverageSuccess(correctEasy, correctHard):
    '''
    Obtain success rate in cognitive task for each subject by converting number of 
    correct responses to a percentage of total trials.
    
    This function treats missed responses as incorrect.

    Parameters
    ----------
    correctEasy -- list containing number of correct responses for each subject in Easy condition
    correctHard -- list containing number of correct responses for each subject in Hard condition

    Returns
    -------
    outcomeEasy -- list; percentage score of each subject in Easy condition
    outcomeHard -- list; percentage score of each subject in Hard condition

    '''
    # Copy input lists so as not to alter them
    outcomeEasy = correctEasy.copy()
    outcomeHard = correctHard.copy()
    
    # Convert number of correct responses into percentage scores
    for i in range(len(outcomeEasy)):
        outcomeEasy[i] = outcomeEasy[i]/40
        outcomeEasy[i] *= 100
    for i in range(len(outcomeHard)):
        outcomeHard[i] = outcomeHard[i]/40
        outcomeHard[i] *= 100
    
    # Return percentages
    return outcomeEasy, outcomeHard

In [ ]:
outcomeEasy, outcomeHard = getAverageSuccess(correctEasy, correctHard)

plt.hist(outcomeEasy)
plt.show()

plt.hist(outcomeHard)
plt.show()

In [ ]:
# Set up list containing array of task outcomes for Easy and Hard condition, respectively
data = [outcomeEasy, outcomeHard]
labels = ['Easy', 'Hard']

# Set up subplots
fig, ax = plt.subplots()
medianprops = dict(linestyle='-', linewidth=2.5, color='firebrick')
bplot = ax.boxplot(data, patch_artist = True, labels = labels, medianprops = medianprops)

# Colour each plot differently
colors = ['lightgreen', 'lightgrey']
for patch, color in zip(bplot['boxes'], colors):
    patch.set_facecolor(color)

# Add labels
plt.xlabel("Experimental Condition", fontsize = 13)
plt.ylabel("Success rate (percentage)", fontsize = 13)
plt.title("Successful responses to cogntive tasks", fontsize = 15)
plt.show()

## Task 3 - Data cleaning

### Steps

1. Sample rate
2. Cleaning before visualisation
3. Cleaning after visualisation
4. Comparing conditions

In [ ]:
pupilDict = convertToDict(pupil)
subject001 = pupilDict['001']

In [ ]:
print(subject001.shape)

In [ ]:
# Identify sample rate

# Select number of columns (-1, as first describes condition) and divide by 6 (seconds of recording)
samplingRate = (subject001.shape[1]-1)/6

print("Sample rate = " + str(int(samplingRate)) + '/sec')

In [ ]:
def makeDataFrames(subject):
    '''
    Convert numpy matrix into pandas dataframe

    Parameters
    ----------
    subject - pupil data matrix of a single subject

    Returns
    -------
    dfDark - data frame of pupillary data within Dark condition
    dfLight - data frame of pupillary data within Light condition

    '''
    # First convert matrix into two dictionaries ('Dark' and 'Light')
    # Start by setting up Dark Dictionary
    subjectDictDark = {}
    
    # Identify number of rows (trials) for each condition
    condition = subject.shape[0]//2
    
    # In Dark Dictionary, set each key to the corresponding trial (column)
    for i in range(condition):
        subjectDictDark['Trial_'+ str(i+1)] = subject001[i,1:]
        
    # Repeat process for Light Dictionary
    subjectDictLight = {}
    for i in range(condition,(condition*2)):
        subjectDictLight['Trial_'+ str((i+1)-condition)] = subject001[i,1:]
    
    # Turn dictionaries into datframes
    dfDark = pd.DataFrame(subjectDictDark)
    dfLight = pd.DataFrame(subjectDictLight)
    
    return dfDark, dfLight

In [ ]:
dfDark, dfLight = makeDataFrames(subject001)
print(dfDark.head(10))
print(dfLight.head(10))

In [ ]:
ffilled = dfDark.ffill(axis = 0)

plt.plot(ffilled.Trial_1)
plt.show()

In [ ]:
plt.plot(dfDark.Trial_1)
plt.show()

In [ ]:
# Extract Trial 1 data (yields a pandas series)
Trial1dark = dfDark.Trial_1.copy()

# Rename this the 'Original', and convert back into dataframe
Trial1dark = Trial1dark.rename('Original')
df1 = Trial1dark.to_frame()

# Add new column into dataframe, named 'Smoothed'
# Perform interpolation, then exponentially-weighted moving average
df1['Smoothed'] = df1['Original'].interpolate()
df1['Smoothed'] = df1['Smoothed'].ewm(span = 200).mean()

In [ ]:
plt.plot(df1['Original'], label = 'Original')
plt.plot(df1['Smoothed'], label = 'Cleaned')
plt.xlabel("Time (miliseconds)", fontsize = 13)
plt.ylabel("Pupil measure", fontsize = 13)
plt.title("Pupil measure during dark screen", fontsize = 15)
plt.legend()
plt.xlim(0, 6000)
plt.show()

In [ ]:
trial1_Dark = dfDark.Trial_1.copy()
t1Dark_array = trial1_Dark.values

def avgFill(a):
    '''
    Own method of filling in data in areas where it is missing. As opposed to ffill or bfill, it results
    in connection that is less aligned with peaks/troughs

    Parameters
    ----------
    a -- array representation of data

    Returns
    -------
    a -- filled array

    '''
    for i in range(len(a)):
        # Identify nan
        if math.isnan(a[i]):
            
            # If gap is at start of timeseries, only a small window can be calculated for a rolling average
            if i < 100:
                window = a[i-10:i-1]
                rollAvg = np.mean(window)
                a[i] = rollAvg
                
            # If the gap occurs later, a larger window can be indexed into
            else:
                window = a[i-100:i-1]
                rollAvg = np.mean(window)
                a[i] = rollAvg
    return a

t1Dark_array = avgFill(t1Dark_array)
plt.xlim(0,6000)
plt.plot(t1Dark_array)
plt.show()

## Detect spikes using Z-score

#### |z(i)| = |(x(i)-μ) / σ|


In [ ]:
def z_score(y):
    '''
    Regular z-score; assess how far a value is from the mean in units of SD

    Parameters
    ----------
    y -- array of data

    Returns
    -------
    z_scores -- array of z-score values
    
    '''
    mean_int = np.mean(y)
    std_int = np.std(y)
    z_scores = (y - mean_int) / std_int
    return z_scores


In [ ]:
z_scores = np.array(abs(z_score(t1Dark_array)))

plt.plot(z_scores)

plt.xlabel('Time (miliseconds' ,fontsize = 13)
plt.ylabel('Z-Score', fontsize = 13)
plt.xlim(left = 0)
plt.xlim(right = 6000)
plt.title('Z-scores', fontsize = 15)

threshold = 3
plt.plot(threshold*np.ones(len(z_scores)), label = 'threshold')
plt.legend()
plt.show()

## Using Modified z-score

#### z(i) = 0.6745(x(i)-M)/ MAD

Here, MAD = median(|x-M|)

In [ ]:
def modified_z_score(y):
    '''
    More robust statistics, using median instead of mean. The multiplier 0.6745 describes the 0.75th quartile
    and MAD refers to Median Absolute Deviation

    Parameters
    ----------
    y -- array of data

    Returns
    -------
    modified_z_scores array of modified z-scores

    '''
    median_int = np.median(y)
    MAD = np.median([np.abs(y - median_int)])
    modified_z_scores = 0.6745 * (y - median_int) / MAD
    return modified_z_scores

In [ ]:
mod_z_scores = np.array(abs(modified_z_score(t1Dark_array)))

plt.plot(mod_z_scores)

plt.xlabel('Time (miliseconds' ,fontsize = 13)
plt.ylabel('Z-Score', fontsize = 13)
plt.xlim(left = 0)
plt.xlim(right = 6000)
plt.title('Modified Z-scores', fontsize = 15)

threshold = 3
plt.plot(threshold*np.ones(len(mod_z_scores)), label = 'threshold')
plt.legend()
plt.show()


## Using Whitaker and Hayes’ modified Z-score based approach for spike detection

#### z(i) = 0.6745 (∇x(i)-M) / MAD

In this manner, distance between consecutive spectrum points is used to calculate Z-score.

##### Original publication: Whitaker, D. A. and K. Hayes (2018). "A simple algorithm for despiking Raman spectra." Chemometrics and Intelligent Laboratory Systems 179: 82-84.

##### Code adapted from: Coca, N., 2021. Removing Spikes from Raman Spectra with Anomaly Detection. [online] Medium. Available at: <https://towardsdatascience.com/removing-spikes-from-raman-spectra-8a9fdda0ac22> [Accessed 10 September 2021].

In [ ]:
def deltaX(a):
    '''
    Calculated ∇x(i) for inputted data

    Parameters
    ----------
    a -- array representation of pupil data

    Returns
    -------
    delta_y array storing ∇x(i) values

    '''
    dist = 0
    delta_y = [] 
    for i in np.arange(len(a)-1):
        dist = a[i+1] - a[i]
        delta_y.append(dist)
    return delta_y

In [ ]:
t1Dark_delta = np.array(deltaX(t1Dark_array))
y_modified_z_score = np.array(np.abs(modified_z_score(t1Dark_delta)))

In [ ]:
plt.plot(y_modified_z_score)

plt.xlabel('Time (miliseconds' ,fontsize = 13)
plt.ylabel('Modified Z-Score with delta x', fontsize = 13)
plt.xlim(left = 0)
plt.xlim(right = 6000)
plt.title('Modified Z-scores', fontsize = 15)

threshold = 5
plt.plot(threshold*np.ones(len(mod_z_scores)), label = 'threshold')
plt.legend()
plt.show()

In [ ]:
spikes = abs(np.array(modified_z_score(t1Dark_delta))) > threshold

plt.xlabel('Time (miliseconds' ,fontsize = 13)
plt.ylabel('Spike present (0/1)', fontsize = 13)
plt.plot(spikes, color = 'red')
plt.title('Spikes: ' + str(np.sum(spikes)), fontsize = 15)
plt.xlim(0, 6000)
plt.show()

In [ ]:
def fixer(y, spikes):
    '''
    Remove spikes using spike data, and replace them with nan. This will allow moving average to restore remaining data points

    Parameters
    ----------
    y -- array to be cleaned
    spikes -- list of spikes (indicates whether spike is present on given index with a boolean)

    Returns
    -------
    y_out -- original data without spike points

    '''
    # Copy y to not overwrite
    y_out = y.copy() 
    for i in np.arange(len(spikes)):
        
        # If we have a spike in position i, remove this data point
        if spikes[i] != False: 
            y_out[i] = np.nan
    return y_out

In [ ]:
yVals = fixer(t1Dark_array, spikes)
plt.plot(t1Dark_array)
plt.show()

plt.plot(yVals)
plt.show()

In [ ]:
series1 = pd.Series(yVals)
series1 = series1.ewm(span = 200).mean()
plt.plot(t1Dark_array, label = 'original')
plt.plot(series1, label = 'cleaned')
plt.legend()
plt.show()

In [ ]:
def plotCleanedData(pdSeries1, pdSeries2):
    '''
    Use all methods above to plot cleaned version for one trial of Dark and one trial of Light

    Parameters
    ----------
    pdSeries1 -- pandas series of one trial (Dark condition)
    pdSeries2 -- pandas series of one trial (Light condition)

    Returns
    -------
    finalSeriesDark -- cleaned version of pupil data from Dark trial
    finalSeriesLight -- cleaned version of pupil data from Light trial

    '''
    # Set custom threshold
    threshold = 5
    
    # Copy original series to not alter them
    a = pdSeries1.values.copy()
    b = pdSeries2.values.copy()
    
    # Define a new function to fill in data; if nan's are present at start, fill only these in using the first real value
    # Full bfill produces suboptimal fill for remaining nan's, however no nan's can be present at the start
    def bfillStart(a):
        if math.isnan(a[0]):
            for i in range(len(a)):
                if math.isnan(a[i]) == False:
                    location = i
                    value = a[i]
                    break
            for i in range(location):
                a[i] = value
        return a
    
    # Only perform this if there is an nan in the 0'th index
    if math.isnan(a[0]):
        a = bfillStart(a)
        
    if math.isnan(b[0]):
        b = bfillStart(b)
    
    # Now perform the average fill, to allow remaining steps to be completed
    a = avgFill(a)
    b = avgFill(b)
    
    # Calculate delta X
    delta_a = np.array(deltaX(a))
    delta_b = np.array(deltaX(b))
    
    # Identify spikes
    spikes1 = abs(np.array(modified_z_score(delta_a))) > threshold
    spikes2 = abs(np.array(modified_z_score(delta_b))) > threshold
    
    # Remove spikes
    yDark = fixer(a, spikes1)
    yLight = fixer(b, spikes2)
    
    # Convert back to series and perform ewm
    finalSeriesDark = pd.Series(yDark)
    finalSeriesDark = finalSeriesDark.ewm(span = 200).mean()
    finalSeriesLight = pd.Series(yLight)
    finalSeriesLight = finalSeriesLight.ewm(span = 200).mean()
    
    # Plot data
    plt.plot(finalSeriesDark, label = 'Dark')
    plt.plot(finalSeriesLight, label = 'Light')
    plt.xlabel("Time (miliseconds)", fontsize = 13)
    plt.ylabel("Pupil measure", fontsize = 13)
    plt.title("Pupil measure during dark and light screen", fontsize = 15)
    plt.legend()
    plt.xlim(0, 6000)
    plt.show()
    
    # Return modified series
    return finalSeriesDark, finalSeriesLight

In [ ]:
DarkCleaned1, LightCleaned1 = plotCleanedData(dfDark.Trial_1, dfLight.Trial_1)
DarkCleaned2, LightCleaned2 = plotCleanedData(dfDark.Trial_2, dfLight.Trial_2)
DarkCleaned3, LightCleaned3 = plotCleanedData(dfDark.Trial_3, dfLight.Trial_3)
DarkCleaned4, LightCleaned4 = plotCleanedData(dfDark.Trial_4, dfLight.Trial_4)
DarkCleaned5, LightCleaned5 = plotCleanedData(dfDark.Trial_5, dfLight.Trial_5)
DarkCleaned6, LightCleaned6 = plotCleanedData(dfDark.Trial_6, dfLight.Trial_6)
DarkCleaned7, LightCleaned7 = plotCleanedData(dfDark.Trial_7, dfLight.Trial_7)
DarkCleaned8, LightCleaned8 = plotCleanedData(dfDark.Trial_8, dfLight.Trial_8)
DarkCleaned9, LightCleaned9 = plotCleanedData(dfDark.Trial_9, dfLight.Trial_9)
DarkCleaned10, LightCleaned10 = plotCleanedData(dfDark.Trial_10, dfLight.Trial_10)

In [ ]:
new_df_Dark = pd.concat([DarkCleaned1, DarkCleaned2, DarkCleaned3, DarkCleaned4, DarkCleaned5, DarkCleaned6, DarkCleaned7, DarkCleaned8, DarkCleaned9, DarkCleaned10], axis = 1)
new_df_Light = pd.concat([LightCleaned1, LightCleaned2, LightCleaned3, LightCleaned4, LightCleaned5, LightCleaned6, LightCleaned7, LightCleaned8, LightCleaned9, LightCleaned10], axis = 1)

print(new_df_Dark.head(10))
print(new_df_Light.head(10))



